# Adicionando funções externas a API da OpenAI

Um grande salto de possibilidades de utilizações únicas da LLMs ocorreu quando a OpenAI lançou a function calling. 
Essa ferramenta permite adicionarmos manualmente funções externas ao modelo que ele, dependendo da situação, poderá utilizar para obter novas informações ou atuar em diversos escopos. Vamos fazer uma breve revisão de como utilizamos funções externas na api da OpenAI.

# Importação Iniciais

In [1]:
import json 

import openai 
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

client = openai.Client()


# Criando função que será adicionada ao modelo

In [4]:
def obter_temperatura_atual(local, unidade='celsius'):
  if 'são paulo' in local.lower():
    return json.dumps(
      {"local": "Sao Paulo", "temperatura": "32", "unidade": unidade}
    )
  elif 'porto alegre' in local.lower():
    return json.dumps(
      {"local": "Porto Alegre", "temperatura": "28", "unidade": unidade}
    )
  else:
    return json.dumps(
      {"local": local, "temperatura": "unknown"}
    )

In [5]:
obter_temperatura_atual("São Paulo")

'{"local": "Sao Paulo", "temperatura": "32", "unidade": "celsius"}'

# Criando descrição da função 

Através dessa descrição o modelo entenderá o que a função faz e como ela pode ser utilizada

In [7]:
tools = [
  {
    'type': 'function',
    'function': {
      'name': 'obter_temperatura_atual',
      'description': 'Obtém a temperatura atual em uma dada cidade',
      'parameters': {
        'type': 'object',
        'properties': {
          'local': {
            'type': 'string',
            'description': 'O nome da cidade, Ex: São Paulo'
          },
          'unidade': {
            'type': 'string',
            'enum': ['celsius', 'fahrenheit']
          }
        },
        'required': ['local'],
      }
    }
  }
]

# Chamando o modelo com a nova ferramenta

Para chamar o modelo com a ferramenta criada, basta passar o argumento tools com uma lista de ferramentas

In [9]:
mensagens = [
  { 'role': 'user', 'content': 'Qual é a temperatura em São Paulo agora?' }
]

resposta = client.chat.completions.create(
  model='gpt-3.5-turbo-0125',
  messages=mensagens,
  tools = tools,
  tool_choice='auto'
)

resposta

ChatCompletion(id='chatcmpl-AfHadqyIzPO4YWDUGcpsuijs5hpUG', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_XDUpzU4aUFB59iaNv8UfiT95', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')]))], created=1734402427, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=20, prompt_tokens=86, total_tokens=106, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

# Analisando a resposta

Podemos perceber que o conteúdo da resposta veio vazio, pois para a pergunta "Qual é a temperatura em São Paulo agora?" ele necessitará chamar a função anes.

In [10]:
mensagem = resposta.choices[0].message
mensagem

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_XDUpzU4aUFB59iaNv8UfiT95', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')])

In [11]:
mensagem.content

#

In [12]:
mensagem.tool_calls

[ChatCompletionMessageToolCall(id='call_XDUpzU4aUFB59iaNv8UfiT95', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')]

In [13]:
tool_call = mensagem.tool_calls[0]
print(tool_call.function.name)
print(tool_call.function.arguments)

obter_temperatura_atual
{"local":"São Paulo"}


# Adicionando resultado da função as mensages

In [15]:
observacao = obter_temperatura_atual(**json.loads(tool_call.function.arguments))
observacao

'{"local": "Sao Paulo", "temperatura": "32", "unidade": "celsius"}'

# Chamando novamente o modelo

In [16]:
mensagens.append(mensagem)
mensagens

[{'role': 'user', 'content': 'Qual é a temperatura em São Paulo agora?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_XDUpzU4aUFB59iaNv8UfiT95', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')])]

In [18]:
mensagens.append({
  'tool_call_id': tool_call.id,
  'role': 'tool',
  'name': tool_call.function.name,
  'content': observacao
})
mensagens

[{'role': 'user', 'content': 'Qual é a temperatura em São Paulo agora?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_XDUpzU4aUFB59iaNv8UfiT95', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')]),
 {'tool_call_id': 'call_XDUpzU4aUFB59iaNv8UfiT95',
  'role': 'tool',
  'name': 'obter_temperatura_atual',
  'content': '{"local": "Sao Paulo", "temperatura": "32", "unidade": "celsius"}'}]

In [19]:
resposta = client.chat.completions.create(
  model='gpt-3.5-turbo-0125',
  messages=mensagens,
  tools=tools,
  tool_choice='auto'
)
resposta

ChatCompletion(id='chatcmpl-AfHn8DZPpIthV39EXO7ZZYg8rp96E', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='A temperatura atual em São Paulo é de 32°C.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1734403202, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=13, prompt_tokens=142, total_tokens=155, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [20]:
resposta.choices[0].message.content

'A temperatura atual em São Paulo é de 32°C.'

# Explorando diferentes perguntas e o parâmetro tool_choice

Através do parâmetro tool_choice é possível forçar o modelo a sempre utiliar uma tool. Vamos ver como ele se comporta para diferentes perguntas modificando a parâmetro.

## Parâmetro "auto"

Assim o modelo define automaticamente se é necessária a utilização de uma função ou não

In [ ]:
mensagens = [
  { 'role': 'user', 'content': 'Qual é a temperatura em São Paulo' }
]

resposta = client.chat.completions.create(
  model='gpt-3.5-turbo-0125',
  messages=mensagens,
  tools=tools,
  tool_choice='auto'
)
mensagem = resposta.choices[0].message
print('Content: ', mensagem.content)
print('Tools: ', mensagem.tool_calls)

Content:  None
Tools:  [ChatCompletionMessageToolCall(id='call_g4yynZ04iX3XAxH9zhpNYJ0W', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')]


In [24]:
mensagens = [
  { 'role': 'user', 'content': 'Oi, tudo bem?' }
]

resposta = client.chat.completions.create(
  model='gpt-3.5-turbo-0125',
  messages=mensagens,
  tools=tools,
  tool_choice='auto'
)
mensagem = resposta.choices[0].message
print('Content: ', mensagem.content)
print('Tools: ', mensagem.tool_calls)

Content:  Olá! Estou aqui para te ajudar. Como posso ajudar você hoje?
Tools:  None


## Parâmetro "none"

Com o parâmetro "none", o modelo não vai utilizar funções.

In [23]:
mensagens = [
  { 'role': 'user', 'content': 'Qual é a temperatura em São Paulo' }
]

resposta = client.chat.completions.create(
  model='gpt-3.5-turbo-0125',
  messages=mensagens,
  tools=tools,
  tool_choice='none'
)
mensagem = resposta.choices[0].message
print('Content: ', mensagem.content)
print('Tools: ', mensagem.tool_calls)

Content:  Vou verificar a temperatura atual em São Paulo para você.
Tools:  None


In [25]:
mensagens = [
  { 'role': 'user', 'content': 'Olá' }
]

resposta = client.chat.completions.create(
  model='gpt-3.5-turbo-0125',
  messages=mensagens,
  tools=tools,
  tool_choice='none'
)
mensagem = resposta.choices[0].message
print('Content: ', mensagem.content)
print('Tools: ', mensagem.tool_calls)

Content:  Olá! Como posso te ajudar hoje?
Tools:  None


## Parâmetro "function"

Podemos fazer o modelo rodar obrigatoriamente a função, passando dentro de um dicionário a função que o modelo deve executar

In [27]:
mensagens = [
  { 'role': 'user', 'content': 'Qual é a temperatura em São Paulo' }
]

resposta = client.chat.completions.create(
  model='gpt-3.5-turbo-0125',
  messages=mensagens,
  tools=tools,
  tool_choice= {'type': 'function', 'function':  {'name': 'obter_temperatura_atual'}}
)
mensagem = resposta.choices[0].message
print('Content: ', mensagem.content)
print('Tools: ', mensagem.tool_calls)

Content:  None
Tools:  [ChatCompletionMessageToolCall(id='call_QYmsaGFYadmR2yTwPbYbsnjz', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')]


In [28]:
mensagens = [
  { 'role': 'user', 'content': 'Olá' }
]

resposta = client.chat.completions.create(
  model='gpt-3.5-turbo-0125',
  messages=mensagens,
  tools=tools,
  tool_choice= {'type': 'function', 'function':  {'name': 'obter_temperatura_atual'}}
)
mensagem = resposta.choices[0].message
print('Content: ', mensagem.content)
print('Tools: ', mensagem.tool_calls)

Content:  None
Tools:  [ChatCompletionMessageToolCall(id='call_R5fOPkkx2GNdTeJRMoAqg7RX', function=Function(arguments='{"local":"São Paulo","unidade":"celsius"}', name='obter_temperatura_atual'), type='function')]
